In [1]:
using QuantumOptics
using PyPlot
using DifferentialEquations
using CollectiveSpins
using JLD2

ERROR: LoadError: UndefVarError: `AbstractNonlinearTerminationMode` not defined in `DiffEqBase`
Stacktrace:
 [1] getproperty(x::Module, f::Symbol)
   @ Base ./Base.jl:42
 [2] top-level scope
   @ ~/.julia/packages/SteadyStateDiffEq/jdIWC/src/solve.jl:70
 [3] include(mod::Module, _path::String)
   @ Base ./Base.jl:557
 [4] include(x::String)
   @ SteadyStateDiffEq ~/.julia/packages/SteadyStateDiffEq/jdIWC/src/SteadyStateDiffEq.jl:1
 [5] top-level scope
   @ ~/.julia/packages/SteadyStateDiffEq/jdIWC/src/SteadyStateDiffEq.jl:9
 [6] include
   @ ./Base.jl:557 [inlined]
 [7] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::String)
   @ Base ./loading.jl:2881
 [8] top-level scope
   @ stdin:6
in expression starting at /Users/remy/.julia/packages/SteadyStateDiffEq/jdIWC/src/solve.jl:70
in expression starting at /Users/remy/.julia/packages/Ste

ErrorException: Failed to precompile DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa] to "/Users/remy/.julia/compiled/v1.11/DifferentialEquations/jl_ZoSRqO".

In [2]:
function H_DE(t, rho)
    E = E0*exp(-1im*ωl*t) + conj(E0)*exp(1im*ωl*t)  # Champ électrique
    H0 = (ω0/2) * Jz  # Hamiltonien sans interaction
    Hint = -d_eg * E * Jx  # Interaction dipôle-champ
    return H0 + Hint + compute_H_SFE(N, bMB, Ω_CS), J, dagger.(J), Γ_CS
end

function H_RWA(t, psi)
    H_RWA = ω0/2*Jz - (Ω_rabi*exp(-1im*ωl*t)*Jp + conj(Ω_rabi)*exp(1im*ωl*t)*Jm)
    return H_RWA + compute_H_SFE(N, bMB, Ω_CS), J, dagger.(J), Γ_CS
end

""" Compute the spin flip exchange hamiltonian """
function compute_H_SFE(N, bMB, Ω_CSC)
    H = SparseOperator(bMB)
    
    for i=1:N, j=1:N
        if i==j
            continue
        end
        
        H += Ω_CSC[i, j] * embed(bMB, i, sp)*embed(bMB, j, sm)
        
    end
    return H
end

compute_H_SFE

In [3]:
# Define geometry of system
Nx,Ny,Nz = [2,2,2]
N = Nx*Ny*Nz
d_xy, d_z = 266., 532. # Optical lattice spacing in nm
e = [0,0,1]   # Quantization axis
Γ0 = 1. # In Hz

λ = 1299.

a,b,c = [d_xy,d_xy,d_z]/λ
geo = geometry.box(a,b,c;Nx=Nx,Ny=Ny,Nz=Nz)
system = SpinCollection(geo, e, gammas=1.)

d_eg = 1.0
E0 = 1.0 + 0im
ωl = 100
ω0 = 101
Ω_rabi = d_eg*E0


Tstep, Tend = 1e-3, 5 # Normalised by Γ0
T = [0:Tstep:Tend;];

UndefVarError: UndefVarError: `geometry` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [10]:
3e8/(1299e-9)

2.3094688221709006e14

In [4]:
Ω_CS = CollectiveSpins.OmegaMatrix(system)
Γ_CS = CollectiveSpins.GammaMatrix(system);

UndefVarError: UndefVarError: `CollectiveSpins` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
# Define Spin basis/operators
b_spin = SpinBasis(1//2)
bMB = CompositeBasis([b_spin for i = 1:N])
sm = sigmam(b_spin)
sp = sigmap(b_spin)
sx = sigmax(b_spin)
sy = sigmay(b_spin)
sz = sigmaz(b_spin)


# Many body operators
Jx = sum([embed(bMB, i, sx) for i = 1:N])
Jy = sum([embed(bMB, i, sy) for i = 1:N])
Jz = sum([embed(bMB, i, sz) for i = 1:N])
Jp = sum([embed(bMB, i, sp) for i = 1:N])
Jm = sum([embed(bMB, i, sm) for i = 1:N])

J = [embed(bMB, i, sm) for i = 1:N];

# Operators of the system. We suppose d // E
d_op = d_eg*sp + conj(d_eg)*sm

# Initial state
Ψ0 = CollectiveSpins.quantum.blochstate(0., 0., N);

UndefVarError: UndefVarError: `d_eg` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
ρt = timeevolution.master_dynamic(T, Ψ0, H_DE)[2]
ρt_RWA = timeevolution.master_dynamic(T, Ψ0, H_RWA)[2];

UndefVarError: UndefVarError: `T` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [7]:
#@save "RO_CS.jld2" ρt ρt_RWA

In [8]:
close("all")
pygui(false)

PyPlot.rc("font", size = 12)

plot(T, real(expect(Jz, ρt))./N, label="Full solution")
plot(T, real(expect(Jz, ρt_RWA))./N, label="RWA")

xlabel(L"t/\tau (AU)", fontsize=16)
ylabel(L"\langle\sigma_z\rangle", fontsize=16)
legend()

#savefig("/Users/remy/Documents/Remy/ENSL_2024_2025/Stage_M2/Rapport_Stage_M2/Figures/RO_CS.pdf", bbox_inches="tight")


gcf()

UndefVarError: UndefVarError: `ρt` not defined in `Main`
Suggestion: check for spelling errors or missing imports.